In [1]:
import fasttext
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gensim

from gensim.models import KeyedVectors
from sklearn import metrics
from sklearn.metrics import hamming_loss, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
def pipe_try(xx_test, yy_test, model):
    print('========================== {} ==========================\n'.format(model))
    print("\t \t \t\t \t------------------")
    print("\t \t \t \t \tF1 Score: ", round(f1_score(yy_test, model.predict(xx_test), average='macro')*100,3),"%")
    print("\t \t \t\t \tAccuracy: ", round(metrics.accuracy_score(yy_test, model.predict(xx_test))*100,3),"%")
    print("    \t \t \t \t       Hamming Loss: ", round(hamming_loss(yy_test, model.predict(xx_test)), 3))
    #print("    \t \t \t \t       ROC_AUC Score: ", round(roc_auc_score(yy_test, model.predict(xx_test))*100, 3), '%')
    print("    \t \t \t \t   Weighted F1 Score: ",round(f1_score(yy_test, model.predict(xx_test), average='weighted')*100,3),"%")
    print("    \t \t \t \t   ---------------------------\n")
    print('================================= Pipeline Specifications ==================================')
    print(model)

In [3]:
embedding_file = '/Users/suhas.kamuni/Downloads/crawl-300d-2M.vec'
df = pd.read_excel('/Users/suhas.kamuni/Downloads/Merged_Training_Set.xlsx')
df.drop(['Platform.1'], axis = 1)

,Review ID,Reviews Content,Converted English,Rating,Start Date,Version,Platform,Source,Month,Predicted Team
0,158994127,Keine vernünftige Struktur wie bei Web Version...,Not a reasonable structure as with web version...,1,21.10.2022,NaN,Android,EasyFeedback,September,People/Homepage
1,158999921,Ich finde vorher war übersicht besser.,I think before was overview better.,2,21.10.2022,NaN,Android,EasyFeedback,September,Design System
2,159009543,Teilweise mühsame Navigation,Partly tedious navigation,3,22.10.2022,NaN,Android,EasyFeedback,September,General
3,159010950,Unpraktische Bedienung,Impractical operation,2,22.10.2022,NaN,Android,EasyFeedback,September,General
4,159033855,"Mich stört, dass laufend irgendwelche Besucher...",It bothers me that some visitors can constantl...,2,22.10.2022,NaN,Android,EasyFeedback,September,Profile
...,...,...,...,...,...,...,...,...,...,...
1788,169331349,Viel zu teuer - möchte kündigen - keine Option...,Way too expensive - want to cancel - no option...,3,25.03.2023 | 12:43,NaN,iOS,EasyFeedback,March,CPAY
1789,170340338,"Zu teuer, zu viel Werbung.","Too expensive, too much advertising.",2,08.04.2023 | 08:33,NaN,Android,EasyFeedback,April,CPAY
1790,171527475,Kosten,Cost,3,25.04.2023 | 19:23,NaN,Android,EasyFeedback,April,CPAY
1791,a03f98f6-bb97-4e90-8885-de666236e05e,Einfach unverschämt diese App. Geld für ein Ja...,Simply outrageous this app. Pull out money for...,1,2023-05-30 00:00:00,23.13.1m,Android,Data.AI,May,CPAY


In [1]:
df

NameError: name 'df' is not defined

In [4]:
# Load vectors directly using Gensim
model = KeyedVectors.load_word2vec_format(embedding_file)

In [5]:
def generate_sentence_vector(sentence):
    words = sentence.split()
    sentence_vector = np.mean([model[word] for word in words if word in model], axis=0)
    return sentence_vector if isinstance(sentence_vector, np.ndarray) else np.zeros((model.vector_size,))

# Generate FastText embeddings for each sentence
sentences_data = df['Converted English']
labels = df['Predicted Team']

In [6]:
# Calculate the sentence vector by averaging the vectors of the words in the sentence
embeddings = np.array([generate_sentence_vector(sentence) for sentence in sentences_data])

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=0)

/Users/suhas.kamuni/Library/Python/3.9/lib/python/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/suhas.kamuni/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [7]:
X_train.shape, y_train.shape

((1434, 300), (1434,))

In [8]:
# Initialize a RandomForest
rf_glove = RandomForestClassifier(n_estimators=100)

# Fit the model
rf_glove.fit(X_train, y_train)

# Predict on test data
y_pred = rf_glove.predict(X_test)

In [9]:
# Print performance details
print(metrics.classification_report(y_test, y_pred))

                      precision    recall  f1-score   support

                 Ads       0.80      0.27      0.40        15
                CPAY       0.00      0.00      0.00         8
Content Architecture       1.00      0.05      0.10        19
       Design System       0.00      0.00      0.00        25
             General       0.43      0.97      0.60       132
            Insights       1.00      0.05      0.09        21
                Jobs       0.33      0.15      0.21        20
 LinkedIn comparison       0.50      0.18      0.27        11
           Messenger       0.00      0.00      0.00        16
        Notification       1.00      0.08      0.14        13
     People/Homepage       0.67      0.58      0.62        31
             Premium       0.50      0.36      0.42        11
             Profile       0.00      0.00      0.00        15
 Reference to Groups       1.00      0.33      0.50        12
              Upsell       0.00      0.00      0.00        10

      

/Users/suhas.kamuni/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/suhas.kamuni/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/suhas.kamuni/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [13]:
pipe_try(X_test, y_test, rf_glove)

========================== RandomForestClassifier() ==========================

	 	 		 	------------------
	 	 	 	 	F1 Score:  22.315 %
	 	 		 	Accuracy:  46.24 %
    	 	 	 	       Hamming Loss:  0.538
    	 	 	 	   Weighted F1 Score:  35.533 %
    	 	 	 	   ---------------------------

================================= Pipeline Specifications ==================================
RandomForestClassifier()


In [11]:
LR_glove = LR_502 = LogisticRegression(C=502, solver='liblinear')
LR_glove.fit(X_train, y_train)
pipe_try(X_test, y_test, LR_glove)

========================== LogisticRegression(C=502, solver='liblinear') ==========================

	 	 		 	------------------
	 	 	 	 	F1 Score:  49.322 %
	 	 		 	Accuracy:  52.368 %
    	 	 	 	       Hamming Loss:  0.476
    	 	 	 	   Weighted F1 Score:  52.37 %
    	 	 	 	   ---------------------------

================================= Pipeline Specifications ==================================
LogisticRegression(C=502, solver='liblinear')


In [12]:
print(metrics.classification_report(y_test, LR_glove.predict(X_test)))

                      precision    recall  f1-score   support

                 Ads       0.40      0.40      0.40        15
                CPAY       0.29      0.25      0.27         8
Content Architecture       0.36      0.47      0.41        19
       Design System       0.42      0.40      0.41        25
             General       0.60      0.51      0.55       132
            Insights       0.46      0.52      0.49        21
                Jobs       0.50      0.65      0.57        20
 LinkedIn comparison       0.55      0.55      0.55        11
           Messenger       0.64      0.44      0.52        16
        Notification       1.00      0.85      0.92        13
     People/Homepage       0.61      0.74      0.67        31
             Premium       0.32      0.55      0.40        11
             Profile       0.44      0.47      0.45        15
 Reference to Groups       0.64      0.75      0.69        12
              Upsell       0.14      0.10      0.12        10

      